### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url_nyt = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19

for page in range(20):
    
    # create query with a page number
    query_url = query_url_nyt + '&page=' + str(page)
    
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    try:
        response = requests.get(query_url)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Check if there are any documents to process
        if response.json()['response']['docs']:
            # Loop through the 'docs' and append each review to the list
            for doc in response.json()['response']['docs']:
                reviews_list.append(doc)
            print(f"Page {page} retrieved and processed.")
        else:
            # Print the page number with no results and break from the loop
            print(f"No results on page {page}, stopping.")
            break

    except requests.RequestException as e:
        print(f"Request failed: {e}")
        break

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    

Page 0 retrieved and processed.
Page 1 retrieved and processed.
Page 2 retrieved and processed.
Page 3 retrieved and processed.
Page 4 retrieved and processed.
Page 5 retrieved and processed.
Page 6 retrieved and processed.
Page 7 retrieved and processed.
Page 8 retrieved and processed.
Page 9 retrieved and processed.
Page 10 retrieved and processed.
Page 11 retrieved and processed.
Page 12 retrieved and processed.
Page 13 retrieved and processed.
Page 14 retrieved and processed.
Page 15 retrieved and processed.
Page 16 retrieved and processed.
Page 17 retrieved and processed.
Page 18 retrieved and processed.
Page 19 retrieved and processed.


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
if len(reviews_list) >= 5:
    # Print the first 5 reviews in a formatted JSON style
    print(json.dumps(reviews_list[:5], indent=4))
else:
    # If there are fewer than 5 reviews, print whatever is available
    print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(
    reviews_list, 
    meta=[
        'web_url',
        'snippet',
        'source',
        ['headline', 'main'],  # Access nested data under 'headline'
        ['headline', 'print_headline'],
        'pub_date',
        ['byline', 'original'],
        'word_count'
    ],
    errors='ignore'  # To ignore missing data in any of the paths
)


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# Extracting the title and creating a new column called 'title'
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
# Function to fix the "keywords" column by converting cells from a list to a string
def fix_keywords_column(df):
    # Apply the extract_keywords function to each cell in the 'keywords' column
    df['keywords'] = df['keywords'].apply(extract_keywords)
    return df
# Call new function:
reviews_df = fix_keywords_column(reviews_df)



In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movie_titles = reviews_df['title'].to_list()

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [11]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in movie_titles:
    
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print("Sleeping for 10 seconds to manage API rate limits")
        time.sleep(10)

    # Add 1 to the request counter
    request_counter += 1

    # Encode the movie title to handle spaces and special characters in URL
    query = requests.utils.quote(title)
    full_url = url + title + tmdb_key_string
    
    # Perform a "GET" request for The Movie Database
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

    try:
         # Execute "GET" request with url
        response = requests.get(full_url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        movie_data = response.json()
        
        # Assume we take the first result as the most relevant if there's any
        if movie_data['results']:
            # Get movie id
            movie_id = movie_data['results'][0]['id']
            details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            details_response = requests.get(details_url)
            # Make a request for a the full movie details
            details_data = details_response.json()
            
            
            # Extract relevant data
            genres = [genre['name'] for genre in details_data.get('genres', [])]
            languages = [lang['english_name'] for lang in details_data.get('spoken_languages', [])]
            countries = [country['name'] for country in details_data.get('production_countries', [])]

            # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
            tmdb_movies_list.append({
                'title': title,
                # Extract the genre names into a list
                'genres': genres,
                # Extract the spoken_languages' English name into a list
                'spoken_languages': languages,
                # Extract the production_countries' name into a list
                'production_countries': countries
            })
            # Confirm to user data is being retrieved for {title}
            print(f"Data retrieved for: {title}")
    
    except requests.RequestException as e:
        print(f"Failed to retrieve data for {title}: {str(e)}")

 
        






Data retrieved for: The Attachment Diaries
Data retrieved for: You Can Live Forever
Data retrieved for: A Tourist’s Guide to Love
Data retrieved for: Other People’s Children
Data retrieved for: One True Loves
Data retrieved for: The Lost Weekend: A Love Story
Data retrieved for: A Thousand and One
Data retrieved for: Your Place or Mine
Data retrieved for: Love in the Time of Fentanyl
Data retrieved for: Pamela, a Love Story
Data retrieved for: In From the Side
Data retrieved for: After Love
Data retrieved for: Alcarràs
Data retrieved for: Nelly & Nadine
Data retrieved for: Lady Chatterley’s Lover
Data retrieved for: The Sound of Christmas
Data retrieved for: The Inspection
Data retrieved for: Bones and All
Data retrieved for: My Policeman
Data retrieved for: About Fate
Data retrieved for: Waiting for Bojangles
Data retrieved for: I Love My Dad
Data retrieved for: A Love Song
Data retrieved for: Alone Together
Data retrieved for: Art of Love
Data retrieved for: The Wheel
Data retrieved 

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
if len(tmdb_movies_list) >= 5:
    # Print the first 5 reviews in a formatted JSON style
    print(json.dumps(tmdb_movies_list[:5], indent=4))
else:
    # If there are fewer than 5 reviews, print whatever is available
    print(json.dumps(tmdb_movies_list, indent=4))

[
    {
        "title": "The Attachment Diaries",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    {
        "title": "You Can Live Forever",
        "genres": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "English",
            "French"
        ],
        "production_countries": [
            "Canada",
            "United States of America"
        ]
    },
    {
        "title": "A Tourist\u2019s Guide to Love",
        "genres": [
            "Romance",
            "Comedy"
        ],
        "spoken_languages": [
            "English",
            "Vietnamese"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "Other People\u2019s Children",
       

In [13]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_df, on='title', how='inner')

In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "", regex=False)


# Display the fixed DataFrame
display(merged_df)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,"Romance, Comedy","English, Vietnamese",United States of America
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,"Drama, Comedy","English, French",France
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Last Face,Drama,"English, Portuguese",United States of America
150,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris,Comedy,French,"Belgium, France"
151,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half,"Drama, Romance",English,Canada
152,https://www.nytimes.com/2017/03/

In [24]:
# Drop "byline.person" column
merged_df.drop(columns = 'byline.person')

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None,A Tourist’s Guide to Love,"Romance, Comedy","English, Vietnamese",United States of America
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None,Other People’s Children,"Drama, Comedy","English, French",France
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None,One True Loves,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None,The Last Face,Drama,"English, Portuguese",United States of America
150,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None,Lost in Paris,Comedy,French,"Belgium, France"
151,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None,The Other Half,"Drama, Romance",English,Canada
152,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None,The Ottoman Lieutenant,"Romance, Drama, War","Turkish, English","Turkey, United States of America"


In [31]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(subset = ['genres', 'spoken_languages', 'production_countries'])
merged_df.reset_index()


,index,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,...,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,...,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,...,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,...,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,"Romance, Comedy","English, Vietnamese",United States of America
3,3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,...,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,"Drama, Comedy","English, French",France
4,4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,...,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,149,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,...,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Last Face,Drama,"English, Portuguese",United States of America
150,150,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,...,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris,Comedy,French,"Belgium, France"
151,151,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half,"Drama, Romance",English,Canada
152,152,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:

In [32]:
# Export data to CSV without the index
merged_df.to_csv(index = False)

'web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries\nhttps://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html,"A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",The New York Times,"subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",,,The Attachment Diaries,,,,By Jeannette Catsoulis,"[{\'firstname\': \'Jeannette\', \'middlename\': None, \'lastname\': \'Catsoulis\', \'qualifier\': None, \'title\': None, \'role\': \'reported\', \'organization\': \'\', \'rank\': 1}]",,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina\nhttps://www.nytimes.com/2023